In [46]:
import numpy as np
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from skimage.io import imread, imshow

# keras imports for building the neural network
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, Flatten
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from keras.utils import np_utils
import tensorflow as tf

In [47]:
# Path to all images
basedir = './dataset'
images_dir = os.path.join(basedir,'image')
labels_filename = 'label.csv'
image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
target_size = None
labels_file = open(os.path.join(basedir, labels_filename), 'r')
lines = labels_file.readlines()
tumor_labels = {line.split(',')[0] : (line.split(',')[1]) for line in lines[1:]}


for i in tumor_labels:
    tumor_labels[i] = tumor_labels[i].replace('\n', '')
    if tumor_labels[i] == "no_tumor":
        tumor_labels[i] = 0
    elif tumor_labels[i] == "meningioma_tumor":
        tumor_labels[i] = 1
    elif tumor_labels[i] == "glioma_tumor":
        tumor_labels[i] = 2
    elif tumor_labels[i] == "pituitary_tumor":
        tumor_labels[i] = 3
        

        
def extract_features_labels():
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            file_name = img_path.split('\\')[-1]

            # load image
            img = imread(img_path, as_gray = True)
            features = np.reshape(img, (512,512,1))
            features.shape, features
            
            if features is not None:
                all_features.append(features)
                all_labels.append(tumor_labels[file_name])

    np_features = np.array(all_features)
    np_labels = np.array(all_labels) 
    return np_features, np_labels



In [11]:
# Path to all images
basedir = './dataset'
images_dir = os.path.join(basedir,'image')
labels_filename = 'label.csv'
image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
target_size = None
labels_file = open(os.path.join(basedir, labels_filename), 'r')
lines = labels_file.readlines()
tumor_labels = {line.split(',')[0] : (line.split(',')[1]) for line in lines[1:]}


for i in tumor_labels:
    tumor_labels[i] = tumor_labels[i].replace('\n', '')
    if tumor_labels[i] == "no_tumor":
        tumor_labels[i] = [1, 0, 0, 0]
    elif tumor_labels[i] == "meningioma_tumor":
        tumor_labels[i] = [0, 0, 0, 1]
    elif tumor_labels[i] == "glioma_tumor":
        tumor_labels[i] = [0, 0, 1, 0]
    elif tumor_labels[i] == "pituitary_tumor":
        tumor_labels[i] = [0, 1, 0, 0]
        

        
def extract_features_labels():
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            file_name = img_path.split('\\')[-1]

            # load image
            img = imread(img_path, as_gray = True)
            features = np.reshape(img, (512,512,1))
            features.shape, features
            
            if features is not None:
                all_features.append(features)
                all_labels.append(tumor_labels[file_name])

    np_features = np.array(all_features)
    np_labels = np.array(all_labels) 
    return np_features, np_labels



In [48]:
X, y = extract_features_labels()
#X = X / 255 
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=42)

# Convert targets into one-hot encoded format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=4)

#y_test = to_categorical(y_test)

#print(y_train)
#print(y_train[3].shape)

In [27]:
# Augmenting the dataset
datagen = ImageDataGenerator(
    rotation_range = 45,
    width_shift_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True)
datagen.fit(x_train)

In [5]:
y

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       ...,
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

In [61]:
CNN_model = Sequential()
CNN_model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1)))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Dropout(0.2))
              
CNN_model.add(Conv2D(32, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Dropout(0.2))

CNN_model.add(Conv2D(64, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Dropout(0.2))

CNN_model.add(Conv2D(128, (3, 3), activation='relu'))
CNN_model.add(MaxPooling2D((2, 2)))
CNN_model.add(Dropout(0.2))
              
CNN_model.add(Flatten())
CNN_model.add(Dense(32, activation='relu'))
#CNN_model.add(Dropout(0.5))
CNN_model.add(Dense(4, activation='softmax'))

In [8]:
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

opt = keras.optimizers.Adam(learning_rate=0.0001)

# model.compile( loss = "categorical_crossentropy", 
#                optimizer = sgd, 
#                metrics=['accuracy']
#              )

In [62]:
CNN_model.compile(loss=keras.losses.categorical_crossentropy, metrics=['accuracy'], optimizer='adam')
print(CNN_model.summary())

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_61 (Conv2D)           (None, 510, 510, 32)      320       
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 255, 255, 32)      0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 253, 253, 32)      9248      
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 126, 126, 32)      0         
_________________________________________________________________
dropout_55 (Dropout)         (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 124, 124, 64)    

In [64]:
# training the model for 10 epochs
CNN_model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_test, y_test))

Train on 2400 samples, validate on 600 samples
Epoch 1/30
2400/2400 [==============================] - 18s 7ms/step - loss: 0.0597 - accuracy: 0.9762 - val_loss: 0.9151 - val_accuracy: 0.8533
Epoch 2/30
2400/2400 [==============================] - 17s 7ms/step - loss: 0.0972 - accuracy: 0.9696 - val_loss: 0.6052 - val_accuracy: 0.8800
Epoch 3/30
2400/2400 [==============================] - 17s 7ms/step - loss: 0.0361 - accuracy: 0.9879 - val_loss: 0.7263 - val_accuracy: 0.8883
Epoch 4/30
2400/2400 [==============================] - 17s 7ms/step - loss: 0.0288 - accuracy: 0.9908 - val_loss: 0.8098 - val_accuracy: 0.8833
Epoch 5/30
2400/2400 [==============================] - 17s 7ms/step - loss: 0.0674 - accuracy: 0.9796 - val_loss: 0.7281 - val_accuracy: 0.8633
Epoch 6/30
2400/2400 [==============================] - 17s 7ms/step - loss: 0.0434 - accuracy: 0.9846 - val_loss: 0.9505 - val_accuracy: 0.8650
Epoch 7/30
2400/2400 [==============================] - 17s 7ms/step - loss: 0.0239